In [32]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import GradientBoostingSurvivalAnalysis,RandomSurvivalForest

from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index
import numpy as np
import pandas as pd

def compare_models(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

    # Initialize the GBST survival regressor
    gbst = GradientBoostingSurvivalAnalysis(n_estimators=100, random_state=20)
    # Fit the model to the training data
    gbst.fit(X_train, y_train)
    # Predict the survival times for the testing data
    survival_times_gbst = gbst.predict(X_test)

    # Create an instance of the RandomSurvivalForest model
    rsf = RandomSurvivalForest(n_estimators=100, random_state=20)
    # Fit the model on the training data
    rsf.fit(X_train, y_train)
    # Predict the survival times for the testing data
    survival_times_rsf = rsf.predict(X_test)

    # Prepare the target variable for CoxPHSurvivalAnalysis
    y_coxph = np.array(list(zip(y_train['event'], y_train['time'])), dtype=[('event', bool), ('time', float)])

    # Create an instance of the CoxPHSurvivalAnalysis model with regularization
    coxph = CoxPHSurvivalAnalysis(alpha=1e-9)
    # Fit the model on the training data
    coxph.fit(X_train, y_coxph)
    # Predict the survival times for the testing data
    survival_times_coxph = coxph.predict(X_test)

    # Compute the concordance index to evaluate the model performance
    c_index_gradient = concordance_index(y_test['time'], -survival_times_gbst, y_test['event'])
    c_index_rf = concordance_index(y_test['time'], -survival_times_rsf, y_test['event'])
    c_index_coxph = concordance_index(y_test['time'], -survival_times_coxph, y_test['event'])

    return {
        "gradient": c_index_gradient,
        "random_survival": c_index_rf,
        "coxph": c_index_coxph
}


In [33]:

data = pd.read_csv('IFR_Extract_with_selected_columns_15-5-23.csv')

obreak_date = pd.to_datetime(data.obreak_date)
datebone = pd.to_datetime(data.datebone)
y = ( abs( datebone - obreak_date))
X = data.drop(["obreak_date","datebone"],axis=1)
selectedColumns = [ 'PatientAge', "PatientGender",'parentbreak', 'alcohol',
                'arthritis', 'diabetes',
                'oralster', 'smoke', 'obreak']
dropList = []
for i in data:
    if data[i].dtypes == 'O':
        dropList.append(data[i].name)
dropList.append("CompletedSurveyId")
dropList.append("PatientId")
X = data.drop(dropList,axis=1)
X.fillna(0,inplace=True)
y = pd.DataFrame({"time":y})

y['event'] = y.time.apply(lambda x: x.days != 0 )
structured_array = y.to_records(index=False)

swapped = pd.DataFrame({
    "event": y.event,
    "time": y.time.apply(lambda x: x.days)
})
(swapped.event).value_counts()
swapped.event = swapped.event.astype(bool)
structured_array = np.rec.array(swapped.to_records(index=False))

mergedBeforeEncoding = pd.concat([X[selectedColumns],swapped],axis=1)
mergedBeforeEncoding

,PatientAge,PatientGender,parentbreak,alcohol,arthritis,diabetes,oralster,smoke,obreak,event,time
0,53,1,0,0,0.0,0.0,0,0,1,True,524
1,85,1,0,0,0.0,1.0,0,0,1,True,2046
2,90,1,0,0,1.0,0.0,0,0,1,True,15455
3,81,1,0,0,0.0,0.0,0,0,1,True,4354
4,60,1,1,0,0.0,1.0,0,0,1,True,2207
...,...,...,...,...,...,...,...,...,...,...,...
795,83,1,4,0,0.0,0.0,4,0,1,True,579
796,60,1,0,0,0.0,0.0,0,0,1,True,5109
797,76,2,0,0,0.0,0.0,0,0,1,True,2125
798,61,1,2,1,0.0,0.0,0,1,1,True,518


In [37]:
import random
# Define the number of synthetic samples
num_samples = 0

# Get the column types for each column in mergedBeforeEncoding
column_types = {}
for column in mergedBeforeEncoding.columns:
    column_types[column] = mergedBeforeEncoding[column].dtype

# Shuffle the feature names
feature_names = list(mergedBeforeEncoding.columns)
random.shuffle(feature_names)

# Initialize an empty DataFrame to store the selected features and their performance
selected_features = pd.DataFrame(columns=["Feature"])

# Create a synthetic data DataFrame with the same columns as mergedBeforeEncoding
synthetic_data = pd.DataFrame(columns=mergedBeforeEncoding.columns)

# Generate synthetic data for each feature
for feature in feature_names:
    column_type = column_types[feature]

    if column_type == bool:
        synthetic_data[feature] = np.random.choice([False, True], size=num_samples)
    else:
        # Sample values from the existing data to maintain the distribution
        existing_data_values = mergedBeforeEncoding[feature].dropna().values
        synthetic_data[feature] = np.random.choice(existing_data_values, size=num_samples)

    synthetic_data[feature] = synthetic_data[feature].astype(column_type)

# Add additional columns to the synthetic data
synthetic_data["obreak"] = 1
synthetic_data["event"] = False
synthetic_data["time"] = 0

augmented_data = pd.concat([mergedBeforeEncoding, synthetic_data], ignore_index=True)



# Store the selected features
selected_features["Feature"] = feature_names

cat_features = ['parentbreak', 'alcohol',
                'oralster', 'smoke'
                # These features were determined to apply minimal impact even
                # 'respdisease', 'hbp','heartdisease',
                # 'ptunsteady', 'wasfractdue2fall', 'cholesterol',
                # 'ptfall', 'shoulder', 'wrist', 'bmdtest_10yr_caroc'
                ]

for feature in cat_features:
    if augmented_data is not None:
        if feature in augmented_data.columns:
            cat_one_hot = pd.get_dummies(augmented_data[feature], prefix=f'{feature}', drop_first=False)
            augmented_data = augmented_data.drop(feature, axis=1)
            augmented_data = augmented_data.join(cat_one_hot)
            
X = augmented_data.drop(['event','time'],axis=1)
y = augmented_data[['event','time']]

y = np.rec.array(y.to_records(index=False))

,PatientAge,PatientGender,arthritis,diabetes,obreak,parentbreak_0,parentbreak_1,parentbreak_2,parentbreak_4,alcohol_0,...,alcohol_3,alcohol_4,oralster_0,oralster_1,oralster_2,oralster_4,smoke_0,smoke_1,smoke_3,smoke_4
0,53,1,0.0,0.0,1,1,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
1,85,1,0.0,1.0,1,1,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
2,90,1,1.0,0.0,1,1,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
3,81,1,0.0,0.0,1,1,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
4,60,1,0.0,1.0,1,0,1,0,0,1,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,83,1,0.0,0.0,1,0,0,0,1,1,...,0,0,0,0,0,1,1,0,0,0
796,60,1,0.0,0.0,1,1,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
797,76,2,0.0,0.0,1,1,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
798,61,1,0.0,0.0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0


In [35]:
import itertools
feature_comparisons = {
    'gradient':'',
    'random_survival':'',
    'coxph':''
    
}
def explore_feature_combinations(X, y, max_iterations=1000):
    best_features = []
    best_performance = 0
    best_model_name = ""

    current_features = set()
    remaining_features = set(X.columns)

    while remaining_features:
        performance_gain = False
        best_gain = 0
        best_feature = None

        for feature in remaining_features:
            features_to_try = current_features | {feature}
            X_subset = X[list(features_to_try)]
            outcome = compare_models(X_subset, y)

            if outcome['gradient'] > best_performance:
                gain = outcome['gradient'] - best_performance
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_model_name = 'gradient'
                    feature_comparisons['gradient'] = outcome['gradient']
                    feature_comparisons['random_survival'] = outcome['random_survival']
                    feature_comparisons['coxph'] = outcome['coxph']

            if outcome['random_survival'] > best_performance:
                gain = outcome['random_survival'] - best_performance
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_model_name = 'random_survival'
                    feature_comparisons['gradient'] = outcome['gradient']
                    feature_comparisons['random_survival'] = outcome['random_survival']
                    feature_comparisons['coxph'] = outcome['coxph']

            if outcome['coxph'] > best_performance:
                gain = outcome['coxph'] - best_performance
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_model_name = 'coxph'
                    feature_comparisons['gradient'] = outcome['gradient']
                    feature_comparisons['random_survival'] = outcome['random_survival']
                    feature_comparisons['coxph'] = outcome['coxph']
            
        if best_feature is not None:
            current_features.add(best_feature)
            remaining_features.remove(best_feature)
            best_performance += best_gain
            performance_gain = True

        if not performance_gain or len(current_features) >= max_iterations:
            break

    return best_model_name, list(current_features), best_performance

# Call the function with a maximum of iterations equal to the total number of features
best_model_name, best_features, best_performance = explore_feature_combinations(X, y, max_iterations=len(X.columns))

print("Best feature combination:", best_features)
print("Highest C-index:", best_performance)
print("Best Model:", best_model_name)
for key,(i,j) in enumerate(feature_comparisons.items()):
    print(i,j)

Best feature combination: ['oralster_0', 'arthritis', 'parentbreak_0', 'parentbreak_1', 'smoke_1', 'oralster_2']
Highest C-index: 0.5519864066672061
Best Model: coxph
gradient 0.5439355934946193
random_survival 0.5471114167812929
coxph 0.5519864066672061


In [36]:
# from sksurv.ensemble import RandomSurvivalForest
# from sklearn.model_selection import train_test_split
# from lifelines.utils import concordance_index


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# # Create an instance of the RandomSurvivalForest model
# rsf = RandomSurvivalForest(n_estimators=100,random_state=20)

# # Fit the model on the training data
# rsf.fit(X_train, y_train)

# # Calculate the baseline performance
# baseline_score = concordance_index(y_test['time'], -rsf.predict(X_test), y_test['event'])

# # Initialize an array to store the feature importances
# feature_importances = np.zeros(X_train.shape[1])

# # Perform feature importance calculation
# for i in range(X_train.shape[1]):
#     # Make a copy of the test set
#     X_permuted = X_test.copy()

#     # Permute the values of the feature at index i
#     X_permuted.iloc[:, i] = np.random.permutation(X_permuted.iloc[:, i])

#     # Calculate the permuted score
#     permuted_score = concordance_index(y_test['time'], -rsf.predict(X_permuted), y_test['event'])

#     # Calculate the feature importance as the difference between the baseline score and permuted score
#     feature_importances[i] = baseline_score - permuted_score

# # Normalize the feature importances
# feature_importances /= np.sum(feature_importances)

# # Print the feature importances
# feature_names = X_train.columns

# #for feature_name, importance in zip(feature_names, feature_importances):
#     #print(f"Feature: {feature_name}, Importance: {importance}")

# df = pd.DataFrame()
# for name, importance in zip(feature_names, feature_importances):
#     df = pd.concat([df, pd.DataFrame({'Feature Name': [name], 'Feature Importance': [importance]})], ignore_index=True)

# df = df.sort_values('Feature Importance', ascending=False)

# df

# # Calculate the c-index on the test set
# c_index = concordance_index(y_test['time'], -rsf.predict(X_test), y_test['event'])
# print("C-index:", c_index)